In this notebook we will build a semantic search engine using ElasticSearch...

# Step 1: Prepare docs

In [1]:

import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [2]:
docs_raw

[{'course': 'data-engineering-zoomcamp',
  'documents': [{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
    'section': 'General course-related questions',
    'question': 'Course - When will the course start?'},
   {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
    'section': 'General course-related questions',
    'question': 'Course - What are the prerequisites for this course?'},
   {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in

In [27]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.041030403226614,
  0.025834161788225174,
  -0.036801841109991074,
  -0.020898321643471718,
  -0.020596304908394814,
  0.009353742003440857,
  -0.003331671468913555,
  -0.009491903707385063,
  0.030117977410554886,
  0.01908210851252079,
  0.012690035626292229,
  -0.017078785225749016,
  -0.0016324761090800166,
  0.12997251749038696,
  0.030969230458140373,
  -0.025823738425970078,
  0.0278230682015419,
  0.025159770622849464,
  -0.0808122381567955,
  -0.0036173474509269,
  -0.008902025409042835,
  0.003404824063181877,
  -0.0230092890560627,
  -0.03404529020190239,
  0.024598615244030952,
  0.013545555993914604,
  -0.025439025834202766,
  0.011951087042689323,
  -0.020540112629532814,
  -0.010077380575239658,
  0.020575348287820816,
  0.0

# Step 2: Create Embeddings using Pretrained Models

Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [28]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")


/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


This code snippet is using the SentenceTransformer library, which is commonly employed for generating embeddings (vector representations) of text.

In [29]:
len(model.encode("This is a simple sentence"))

768

- The output 768 means that the generated vector embedding for the input sentence has 768 dimensions.

- This dimension size comes from the underlying model architecture (all-mpnet-base-v2), as it generates a 768-dimensional dense vector for each input text.

In [30]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.041030403226614,
  0.025834161788225174,
  -0.036801841109991074,
  -0.020898321643471718,
  -0.020596304908394814,
  0.009353742003440857,
  -0.003331671468913555,
  -0.009491903707385063,
  0.030117977410554886,
  0.01908210851252079,
  0.012690035626292229,
  -0.017078785225749016,
  -0.0016324761090800166,
  0.12997251749038696,
  0.030969230458140373,
  -0.025823738425970078,
  0.0278230682015419,
  0.025159770622849464,
  -0.0808122381567955,
  -0.0036173474509269,
  -0.008902025409042835,
  0.003404824063181877,
  -0.0230092890560627,
  -0.03404529020190239,
  0.024598615244030952,
  0.013545555993914604,
  -0.025439025834202766,
  0.011951087042689323,
  -0.020540112629532814,
  -0.010077380575239658,
  0.020575348287820816,
  0.0

The following code creates dense vector embeddings for the text in a list of documents using a pre-trained model and stores those vectors in each document:

In [9]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the title into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [31]:
operations[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.041030403226614,
  0.025834161788225174,
  -0.036801841109991074,
  -0.020898321643471718,
  -0.020596304908394814,
  0.009353742003440857,
  -0.003331671468913555,
  -0.009491903707385063,
  0.030117977410554886,
  0.01908210851252079,
  0.012690035626292229,
  -0.017078785225749016,
  -0.0016324761090800166,
  0.12997251749038696,
  0.030969230458140373,
  -0.025823738425970078,
  0.0278230682015419,
  0.025159770622849464,
  -0.0808122381567955,
  -0.0036173474509269,
  -0.008902025409042835,
  0.003404824063181877,
  -0.0230092890560627,
  -0.03404529020190239,
  0.024598615244030952,
  0.013545555993914604,
  -0.025439025834202766,
  0.011951087042689323,
  -0.020540112629532814,
  -0.010077380575239658,
  0.020575348287820816,
  0.0

# Step 3: Setup ElasticSearch connection

In [32]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '522e3515f5ca', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'rtHIEG84QDSEE-2id0G2tw', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

# Step 4: Create Mappings and Index

- Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

- Each document is a collection of fields, which each have their own data type.

- We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

The following code sets up an Elasticsearch index named "course-questions" with specific settings and mappings, which includes the configuration for storing dense vector embeddings for semantic search:

In [33]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [34]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True) # This line deletes any existing index with the name "course-questions" if it exists.
es_client.indices.create(index=index_name, body=index_settings) # This creates a new index called "course-questions" using the index_settings defined earlier.

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

The above code is defining a schema for an Elasticsearch index. The term "schema" in this context refers to the structure of the data that the index will store and how Elasticsearch should interpret different fields. This schema is defined through settings and mappings.

Key Aspects of the Schema:

### Settings:

- Define configurations for how Elasticsearch should manage the index, such as the number of shards and replicas.
Settings impact performance and scalability.

### Mappings:

- Define the structure and data types of the documents that will be stored in the index.
- Each field in the documents (like text, section, question, course, text_vector) is mapped to a specific type (e.g., text, keyword, dense_vector).
- Mappings ensure that Elasticsearch knows how to handle different types of fields, whether they require full-text analysis, exact matching, or vector-based similarity.

# Step 5: Add documents into index

The following code snippet indexes documents into an Elasticsearch index called `index_name`.

In [35]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

# Step 6: Create end user query

This code performs a k-nearest neighbors (k-NN) search using vector embeddings in Elasticsearch, aiming to find the top 5 most similar documents to a given `search_term`:

In [36]:
# Search Term and Vectorization:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [37]:
# k-NN Query Definition:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

Above snippet:

- Specifies a k-nearest neighbors (k-NN) search on the text_vector field.
- query_vector: The vector representation of the search term.
- k: 5: Requests the top 5 most similar results.
- num_candidates: 10000: Searches through up to 10,000 candidates to find the top 5 most similar.

In [38]:
# Search Execution and retrieve/display hits
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'rdpLZ5IBBUdtJh1fMZbV',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'wNpLZ5IBBUdtJh1fpZnV',
  '_score': 0.61347336,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

The output above is a list of dictionaries representing the top 5 search results, each with:

- _index: The index name (course-questions).
- _id: A unique ID for the document in Elasticsearch.
- _score: The similarity score between the search term's vector and the document's vector (higher means more similar).
- _source: Contains the original fields like question, course, section, and text.

# Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

The follwoing code performs a semantic search with filtering using Elasticsearch, where the search is *restricted to a specific section* and retrieves the most similar results based on a query vector.

In [39]:

# Note: I made a minor modification to the query shown in the notebook here
# (compare to the one shown in the video)
# Included "knn" in the search query (to perform a semantic search) along with the filter  
knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000 # Considers up to 10,000 candidates to find the top matches.
}

In [40]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"}, # restricting to this section only
    },
    knn=knn_query,
    size=5
)

In [41]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'rdpLZ5IBBUdtJh1fMZbV',
  '_score': 11.614713,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
    -0.015976183116436005,
    0.008629541844129562,
    -0.02447899058461

Now restricting the course title, not the section title:

In [46]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"}, # restricting to this section only
    },
    knn=knn_query,
    size=5,
    explain=True # add an explanation of the scores
)

In [47]:
response["hits"]["hits"]

[{'_shard': '[course-questions][0]',
  '_node': 'LMsQWlyTTTOeIrMOU60Ghw',
  '_index': 'course-questions',
  '_id': 'rdpLZ5IBBUdtJh1fMZbV',
  '_score': 1.4937059,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.026965461671352386,
    -0.000626126304268837,
    -0.01662949100136757,
    0.05285150930285454,
    0.05476527288556099,
    -0.03133990615606308,
    0.029942581430077553,
    -0.04808562621474266,
    0.04467551037669182,
    0.005839474033564329,
    0.016233040019869804,
    0.012001154012978077,
    -0.031222281977534294,
    0.016600528731942177,
    -0.04886901378631592,
    -0.06496307998895645,
    0.046434223651885986,
    -0.009297756478190422,
    -0.0642528235912323,
    -0.01373267825692892,
